In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
if not os.getcwd().endswith('CIoTS'):
    os.chdir('../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from CIoTS import *

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Helper: check if data exists

In [10]:
import os.path

def check_setups(setups, data_path):
    return not missing_setups(setups, data_path)

def missing_setups(setups, data_path):
    missing = []
    for dim, in_edges, tau, autocorr, _, run in setups:
        if not os.path.isfile(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle'):
            missing.append((dim, in_edges, tau, autocorr))#, run))
    return list(set(missing))

In [14]:
sorted(missing_setups(setups, 'notebooks/ICML/icml_data/'))

[(10, 3, 5, True),
 (10, 3, 10, False),
 (10, 3, 10, True),
 (10, 3, 15, False),
 (10, 3, 15, True),
 (10, 3, 20, False),
 (10, 3, 20, True),
 (10, 4, 5, False),
 (10, 4, 5, True),
 (10, 4, 10, False),
 (10, 4, 10, True),
 (10, 4, 15, False),
 (10, 4, 15, True),
 (10, 4, 20, False),
 (10, 4, 20, True)]

# Execution

In [12]:
from itertools import product

dimensions = [3]
incoming_edges = [2, 3, 4]
taus = [5, 10, 15, 20]
autocorrs = [False, True]
data_length = [10000]
runs = range(10)

setups = list(product(dimensions, incoming_edges, taus, autocorrs, data_length, runs))

## 1. Compute $F_1$ for known $\tau$

PC1 iterative vs. PC iterative

Also check for $\tau - k$ and $\tau + k$ to visualize importance of $\tau$ estimation

In [6]:
import pickle

data_path = 'notebooks/ICML/icml_data/'
results_path = 'notebooks/ICML/icml_results/'
results = pd.DataFrame()
k = 2

algorithms = [(pc_incremental, 'PC incremental'),
              (pc_incremental_extensive, 'PC extensive'),
              (pc_incremental_pc1, 'PC1 incremental')]

In [ ]:
if not check_setups(setups, data_path):
    print('Missing setups:')
    print(missing_setups(setups, data_path))

for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    
    df_dict = {'dimension': dim, 'max time lag': tau, 'incoming edges': in_edges, 'run': run, 'autocorr': autocorr}
    for algorithm, name in algorithms:
        _, graphs, _, _, _ = algorithm(partial_corr_test, generator.ts, max_p=tau+k, 
                                       use_stopper=False, alpha=0.01, verbose=True)
        for offset in range(-k, k+1):
            f1_score = evaluate_edges(generator.graph, graphs[tau+offset])['f1-score']
            df_dict[name + '_f1' + (f'+{offset}' if offset>=0 else str(offset))] = f1_score

    results = results.append(df_dict, ignore_index=True)

results.to_csv(results_path + f'experiment1-Alpha001.csv', index=False)

Add results for non-iterative

In [ ]:
results = pd.read_csv(results_path + f'experiment1-Alpha001.csv')
for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    
    df_dict = {'dimension': dim, 'max time lag': tau, 'incoming edges': in_edges, 'run': run, 'autocorr': autocorr}
    name = 'PC1 non-incremental'

    for offset in range(-k, k+1):
        _, graphs, _, _, _ = pc_incremental_pc1(partial_corr_test, generator.ts, max_p=tau+offset,
                                                steps=tau+offset,
                                                use_stopper=False, alpha=0.01, verbose=True)
        assert len(graphs) == 1
        f1_score = evaluate_edges(generator.graph, graphs[tau+offset])['f1-score']
        df_dict[name + '_f1' + (f'+{offset}' if offset>=0 else str(offset))] = f1_score

    results = results.append(df_dict, ignore_index=True)
results.to_csv(results_path + f'experiment1-Alpha001_incl-non-inc.csv', index=False)

In [8]:
results = pd.read_csv(results_path + f'experiment1-Alpha001_incl-non-inc.csv')

In [24]:
names = ['PC incremental', 'PC extensive', 'PC1 incremental', 'PC1 non-incremental']

In [25]:
names

['PC incremental', 'PC extensive', 'PC1 incremental', 'PC1 non-incremental']

In [30]:
results.groupby(['dimension', 'incoming edges', 'autocorr', 'max time lag', 'run']
               ).min().groupby(['dimension', 'incoming edges', 'autocorr', 'max time lag']).mean(
                )[[n + '_f1+1' for n in names]]

PC incremental_f1+1  \
dimension incoming edges autocorr max time lag                        
3.0       2.0            0.0      5.0                      0.694929   
                                  10.0                     0.862987   
                                  15.0                     0.942105   
                                  20.0                     0.860865   
                         1.0      5.0                      0.641513   
                                  10.0                     0.617101   
                                  15.0                     0.716766   
                                  20.0                     0.768450   
          3.0            0.0      5.0                      0.776996   
                                  10.0                     0.667173   
                                  15.0                     0.776260   
                                  20.0                     0.656708   
                         1.0      5.0                      0.646031   
                                  10.0                     0.766536   
                                  15.0                     0.657270   
                                  20.0                     0.613032   
          4.0            0.0      5.0                      0.634991   
                                  10.0                     0.650663   
                                  15.0                     0.676684   
                                  20.0                     0.733504   
                         1.0      5.0                      0.576908   
                                  10.0                     0.518046   
                                  15.0                     0.497864   
                                  20.0                     0.507111   

                                                PC extensive_f1+1  \
dimension incoming edges autocorr max time lag                      
3.0       2.0            0.0      5.0                    0.694602   
                                  10.0                   0.864719   
                                  15.0                   0.942105   
                                  20.0                   0.855151   
                         1.0      5.0                    0.626479   
                                  10.0                   0.547913   
                                  15.0                   0.725814   
                                  20.0                   0.784632   
          3.0            0.0      5.0                    0.788490   
                                  10.0                   0.639247   
                                  15.0                   0.785744   
                                  20.0                   0.670211   
                         1.0      5.0                    0.637137   
                                  10.0                   0.767698   
                                  15.0                   0.620337   
                                  20.0                   0.622075   
          4.0            0.0      5.0                    0.634991   
                                  10.0                   0.667719   
                                  15.0                   0.696288   
                                  20.0                   0.729221   
                         1.0      5.0                    0.582699   
                                  10.0                   0.547020   
                                  15.0                   0.565410   
                                  20.0                   0.500788   

                                                PC1 incremental_f1+1  \
dimension incoming edges autocorr max time lag                         
3.0       2.0            0.0      5.0                       0.688177   
                                  10.0                      0.880136   
                                  15.0                      0.922711   
                                  20.0        

## 2. Compute estimate $\hat{\tau}$ for unknown $\tau$

Test VAR estimation vs. Incremental BIC (for $PC_1$) vs. Correltation test based

In [ ]:
import pickle

data_path = 'notebooks/ICML/icml_data/'
results_path = 'notebooks/ICML/icml_results/'
results = pd.DataFrame()

if not check_setups(setups, data_path):
    print('Mising setups:')
    print(missing_setups(setups, data_path))

for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    
    df_dict = {'dimension': dim, 'max time lag': tau, 'incoming edges': in_edges, 'run': run, 'autocorr': autocorr}
    algorithms = [
        (pc_incremental_pc1, {
            'indep_test': partial_corr_test,
            'ts': generator.ts,
            'max_p': 2*tau,
            'stopper': ICStopper(dim, patiency=2, ic='bic'),
            'verbose': True}, lambda r: r[3].best_tau, 'PC1 incremental: BIC'),
        (pc_incremental_pc1, {
            'indep_test': partial_corr_test,
            'ts': generator.ts,
            'max_p': 2*tau,
            'stopper': CorrStopper(dim, patiency=1, max_tau=2*tau),
            'verbose': True}, lambda r: r[3].best_tau, 'PC1 incremental: Correlation'),
        (var_order_select, {
            'ts': generator.ts, 
            'max_p': 2*tau, 
            'ics': ['bic']}, lambda r: r[0]['bic'][0], 'VAR: BIC'),
        (pc_incremental, {
            'indep_test': partial_corr_test,
            'ts': generator.ts,
            'max_p': 2*tau,
            'stopper': ICStopper(dim, patiency=2, ic='bic'),
            'verbose': True}, lambda r: r[3].best_tau, 'PC incremental: BIC'),
        (pc_incremental_extensive, {
            'indep_test': partial_corr_test,
            'ts': generator.ts,
            'max_p': 2*tau,
            'stopper': ICStopper(dim, patiency=2, ic='bic'),
            'verbose': True}, lambda r: r[3].best_tau, 'PC extensive: BIC')
    ]
    for algorithm, params, result_func, name in algorithms:
        result = algorithm(**params)
        df_dict[name + '_tau'] = result_func(result)
    results = results.append(df_dict, ignore_index=True)

results.to_csv(results_path + f'experiment2-Alpha001.csv', index=False)

## 3. Evaluate each iteration

In [ ]:
import pickle

data_path = 'notebooks/ICML/icml_data/'
results_path = 'notebooks/ICML/icml_results/'
results = pd.DataFrame()

algorithms = [(pc_incremental, 'PC incremental'),
              (pc_incremental_extensive, 'PC extensive'),
              (pc_incremental_pc1, 'PC1 incremental')]

if not check_setups(setups, data_path):
    print('Mising setups:')
    print(missing_setups(setups, data_path))   
    
for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    max_tau = 2 * tau
    
    df_dict = {'dimension': [dim]*max_tau, 'max time lag': [tau]*max_tau, 'incoming edges': [in_edges]*max_tau, 
               'run': [run]*max_tau, 'autocorr': [autocorr]*max_tau, 'tau estimate': list(range(1, max_tau+1))}
    
    for algorithm, name in algorithms:
        _, graphs, _, stopper, _ = algorithm(partial_corr_test, generator.ts, max_p=max_tau, 
                                             use_stopper=False, alpha=0.01, verbose=True)

        confusion, confusion_delta = evaluate_edge_deletion(generator.graph,
                                                            [{'graph': graphs[t], 'p_iter': t} 
                                                             for t in range(1, max_tau+1)],
                                                            dim)
        added_edges = (confusion_delta['tp'] + confusion_delta['fp']).tolist()
        f1_scores = [evaluate_edges(generator.graph, graphs[t])['f1-score'] for t in range(1, max_tau+1)]
        bics = [stopper.scores()[t] for t in range(1, max_tau+1)]
        
        eval_dict = {name + '_tn': confusion['tn'].tolist(), name + '_fp': confusion['fp'].tolist(),
                     name + '_tp': confusion['tp'].tolist(), name + '_fn': confusion['fn'].tolist(),
                     name + '_f1': f1_scores, name + '_bics': bics, name + '_added_edges': added_edges}
        df_dict.update(eval_dict)
        
    df = pd.DataFrame(df_dict)
    results = results.append(df, ignore_index=True)

results.to_csv(results_path + f'experiment3-Alpha001.csv', index=False)

# Visualization

## 1. Visualize $F_1$ for known $\tau$

In [ ]:
from functools import reduce
import sys
eps = sys.float_info.epsilon

def plot_f1_groupedbarchart(df_dict, title):
    assert len(df_dict) > 0
    #assert reduce(lambda x,y: np.all(x == y), map(lambda e:e.columns, grouped_result.values()))
    bar_labels = [f'$\\tau + {c[1]}$' if c[1]>=0 else f'$\\tau - {abs(c[1])}$'
                  for c in next(iter(grouped_result.values())).columns]
    
    fig, ax = plt.subplots(figsize=(20,10))
    ind = np.arange(len(bar_labels))
    width = 0.7 / len(df_dict)
    
    plots = []
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for w_idx, (algo, f1s) in enumerate(df_dict.items()):
        p = ax.bar(ind + w_idx*width + (width / 2), np.mean(f1s, axis=0), width, 
                   color=color_cycle[w_idx % len(color_cycle)],
                   bottom=0, yerr=np.std(f1s, axis=0))
        plots.append(p[0])

    ax.set_title(title)
    ax.set_xticks(ind + (len(df_dict) * width) / 2)
    ax.set_xticklabels(bar_labels)

    ax.legend(plots, df_dict.keys())
    ax.autoscale_view()

    plt.show()

In [ ]:
results_path = 'notebooks/ICML/icml_results/'
algorithms = [(pc_incremental, 'PC incremental'),
              (pc_incremental_extensive, 'PC extensive'),
              (pc_incremental_pc1, 'PC1 incremental'),
              (None, 'PC1 non-incremental')]

results = pd.read_csv(results_path + 'experiment1.csv')
for group, result in results.groupby(['max time lag', 'autocorr']): # Group by ['dimension', 'incoming edges', 'autocorr'] as well?
    tau = int(group[0])
    autocorr = float(group[1])
    grouped_result = {}
    for _, name in algorithms:
        algo_cols = [col for col in result.columns if col[:col.rfind('_')] == name]
        df = result[algo_cols].rename(lambda c:('f1', int(c.split('_')[-1][2:])), axis='columns')
        grouped_result[name] = df[sorted(df.columns)]
    plot_f1_groupedbarchart(grouped_result, title=f'$F_1$ score for known $\\tau$ = {tau}, autocorr = {autocorr}')

## 2. Visualize estimate $\hat{\tau}$ for unknown $\tau$

In [ ]:
from math import floor

visualize = [(3, 3, 0), (3, 3, 0.75), (3, 4, 0)]
names = ['PC1 incremental: BIC', 'PC1 incremental: Correlation',
         'VAR: BIC', 'PC incremental: BIC', 'PC extensive: BIC']

width = 0.4
results_path = 'notebooks/ICML/icml_results/'
cols = [name + '_tau' for name in names]

results = pd.read_csv(results_path + 'experiment2-Alpha001.csv')
true_taus = np.unique(results['max time lag'])

for group, result in results.groupby(['dimension', 'incoming edges', 'autocorr']): 
    
    if group not in visualize:
        continue
    
    dim = int(group[0])
    in_edges = int(group[1])
    autocorr = float(group[2])
    
    colors = plt.cm.CMRmap(np.linspace(0,1,len(names)+2))
    plt.figure(dpi=200, figsize=(15, 8))
    
    
    for tau in true_taus:
        start_x = tau - (width+0.1)*(len(names))/2
        end_x = tau + (width+0.1)*(len(names))/2
        y = tau
        plt.hlines(y, start_x, end_x, colors='grey')
    
    bps = []
    for i, name in enumerate(names):
        positions = [tau + (width+0.1)*(i - (len(names)-1)/2) for tau in true_taus]
        arr = [result.loc[result['max time lag']==tau, cols[i]] for tau in true_taus]
        bp = plt.boxplot(arr, positions=positions, widths=width, showfliers=False,
                         whiskerprops={'color': colors[i+1]}, boxprops={'color': colors[i+1]},
                         capprops={'color': colors[i+1]}, medianprops={'color': 'black'})
        bps.append(bp)
    
    plt.xlabel('true $\\tau$')
    plt.ylabel('estimated $\\hat{\\tau}$')
    plt.xticks(true_taus, true_taus)
    plt.title(f'$\\tau$ estimation for dimensionality={dim}, incoming edges={in_edges}, autocorr={autocorr}')
    plt.xlim(min(true_taus)-(width+0.2)*len(names)/2, max(true_taus)+(width+0.2)*len(names)/2)
    
    plt.legend([bps[i]['whiskers'][0] for i in range(len(names))], names)
    plt.show()

## 3. Visualize iterations

In [ ]:
from math import floor

visualize = [(10, 3, 3, 0.75)]
names = ['PC1 incremental', 'PC extensive', 'PC incremental']
prop = 'fn'

width = 0.4
results_path = 'notebooks/ICML/icml_results/'

results = pd.read_csv(results_path + 'experiment3-Alpha001.csv')
true_taus = np.unique(results['max time lag'])

for group, result in results.groupby(['max time lag', 'dimension', 'incoming edges', 'autocorr', 'run']): 
    
    tau = int(group[0])
    dim = int(group[1])
    in_edges = int(group[2])
    autocorr = float(group[3])
    run = int(group[4])
    
    if (tau, dim, in_edges, autocorr) not in visualize:
        continue
    
    colors = plt.cm.CMRmap(np.linspace(0,1,len(names)+2))
    plt.figure(dpi=200, figsize=(15, 8))
    
    x = result['tau estimate']
    for i, name in enumerate(names):
        y = result[name + '_' + prop]
        plt.plot(x, y, color=colors[i+1], label=name)
    
    plt.xlabel('iteration $\\tau$')
    plt.ylabel(prop)
    plt.title(f'{prop} for dimensionality={dim}, incoming edges={in_edges}, autocorr={autocorr}, run={run}')
    
    plt.legend()
    plt.show()